In [1]:
import os,sys,inspect
from ipywidgets import interact, interactive, fixed, interact_manual, Button
from IPython.display import display, Markdown, Latex, Javascript, clear_output
import emoji
import time
import ipywidgets as widgets
import ipython_blocking
import numpy as np
import time
from tqdm import tqdm_notebook
import functools
import configparser
import webbrowser
import shutil

parentdir = os.path.abspath('../../../pymodules')

sys.path.insert(0,parentdir)
#from connect_device import connect
from connect_device_v2.vizualizate import vizulizated_finds_devices, advanced_search_device,DeviceBundle
from connect_device_v2.save_file import SaveFilesButton
from connect_device_v2.connect import connect_to_device
from connect_device_v2.find_device import FindDevice
from connect_device_v2.auxiliary_commands import check_error,disable_all_correctors,enable_ifbf_for_all_segment
import pandas as pd
from datetime import datetime
button = widgets.Button(
                description='Continue',
                disabled=False,
                button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
                tooltip='Click me',
                icon='play'        
                )

testname="Проверка устойчивости к механическому воздействию"
display(Markdown('### 💠 {}'.format(testname)))

### 💠 Проверка устойчивости к механическому воздействию

In [2]:
from connect_device_v2.find_device import FindDevice
box = FindDevice()
display(box)

Выбранная конфигурация
filter_if:10 kHz
point_value:200
athenuator_value:10
cap_value:6554


<IPython.core.display.Javascript object>

In [11]:
button = box.select_device_continue_button

In [12]:
%blockrun button

In [13]:
button1 = box.select_config_continue_button

In [14]:
%blockrun button1

In [15]:
inst = box.selected_device

In [16]:
inst.write("SYST:PRES")

(10, <StatusCode.success: 0>)

In [17]:
result_config =  box.config

In [18]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jun 21 09:28:15 2021

@author: kuzma.m
"""

import pyvisa as visa

#функция декоратор
def command_wrapper(f):
    def wrapped(inst, command):
        # print('До функции')
        try:
            if key == "debug":
                print(command)
        except:
            pass
        response = f(inst, command)
        # print('После функции')
        error = inst.query('SYST:ERR?')
        if error != '0, No error':
            print(error)
        return response

    # print('декорируем', f)
    return wrapped

#запрос данных
@command_wrapper
def query(inst, command):
    result = inst.query(command)
    return result

@command_wrapper
def query_ascii_values(inst, command):
    result = inst.query_ascii_values(command)
    return result

#запрос данных
@command_wrapper
def write(inst, command):
    inst.write(command)

#Функция конвертации множителей kGz,MGh,GHz в герцы
def HzConvertor(n,k):
    if k == "kHz" or k == "кГц":
        return int(n*1e+3)
    elif k == "MHz" or k == "МГц":
        return int(n*1e+6)
    elif k == "GHz" or k == "ГГц":
        return int(n*1e+9)
    else: return -1

#Функция однократного росчерка
def singlescan(inst):
    write(inst, f'TRIG:SOUR BUS')
    write(inst, f'INIT:CONT OFF')
    write(inst, f'INIT')
    write(inst, f'TRIG:SING')
    query(inst, f'*OPC?')

In [19]:
value_dict = {'table':{}}

In [20]:
out = widgets.HTML()

In [21]:
def parse_config(config: dict):
    new_config = config.copy()
    
    if_split = config['DEVICE']['filter_if'].split(" ")
    if_value = f'{HzConvertor(int(if_split[0]),if_split[1])}'
    
    
    at_value = config['DEVICE']['athenuator_value']
    poin_value = config['DEVICE']['point_value']
    cap_value = config['DEVICE']['cap_value']
    
    
    new_config['DEVICE']['if_value']=if_value
    new_config['DEVICE']['athenuator_value']=at_value
    new_config['DEVICE']['point_value'] = poin_value
    new_config['DEVICE']['cap_value'] = cap_value
    return new_config
    

In [22]:
new_config=parse_config(dict(result_config))

In [23]:
def param(new_config):
    #Пресет
    write(inst, f'SYST:PRES')
    
    #Устанавливаем линейное сканирование
    write(inst, f'SENS:SWE:TYPE LIN')
    
    #Запрашиваем полный диапазон частот прибора и устанавливаем
    #freq_max = query(CMT, f'SERV:SWE:FREQ:MAX?')
    #freq_min = query(CMT, f'SERV:SWE:FREQ:MAX?')
    
    
    #Attenuator control – вкл. 
    write(inst, f'SERV:RFCTL:POW:STAT 1')
    write(inst, f'SERV:RFCTL:POW:ATT {new_config["DEVICE"]["athenuator_value"]}')

    #Установить фильтр ПЧ
    write(inst, f'SENS:BWID {new_config["DEVICE"]["if_value"]}')
    time.sleep(1) 

    #установить Код ЦАП.
    write(inst, f'SERV:RFCTL:POW:DAC {new_config["DEVICE"]["cap_value"]}')
    
    #установить кол-во точек.
    write(inst, f'SENS:SWE:POIN {new_config["DEVICE"]["point_value"]}')


In [24]:
def setport():
    ports = query(inst,f'SERV:PORT:COUN?')
    write(inst, f'CALC1:PAR:COUN {ports}')
    for port in range(1,int(ports) + 1):
        write(inst, f'CALC1:PAR{port}:DEF S{port}{port}')

In [25]:
def normalize():
    coun = query(inst,f'CALC1:PAR:COUN?')
    for trace in range(1,int(coun) + 1):
        write(inst,f'CALC:PAR{trace}:SEL')
        write(inst,f'CALC:MATH:MEM')
        write(inst,f'CALC:MATH:FUNC DIV')
    

In [26]:
def gridbutton(count):
    row = 0
    port_count = 0
    
    if count%6 == 0:
        row_1 = count//6
    else:
        row_1 = count//6+1
    c = 6
    grid = widgets.GridspecLayout(row_1 * 3, 6)
    for i in range(row_1):
        if i == row_1 - 1:
            c = count - c * i
        for j in range(c):
            port_count += 1
            g1 = widgets.Button(description=f'До воздействия {port_count}')
            g1.on_click(functools.partial(on_button_clicked, rs_=f'До  {port_count}'))

            g2 =  widgets.Button(description=f'После воздействия {port_count}')
            g2.on_click(functools.partial(on_button_clicked, rs_=f'После {port_count}'))
            grid[row + 0, j] = widgets.Button(description=f'порт {port_count}')
            grid[row + 1, j] = g1
            grid[row + 2, j] = g2
        row += 3
    display(grid)


In [27]:
#Устанавливаем параметры и выставляем порты
param(new_config)
setport()

In [28]:
#кнопка для выполнения нормализации
button_normalize = widgets.Button(description='Нормализировать', 
                        button_style='success' # 'success', 'info', 'warning', 'danger', ''
                        )
display(button_normalize)

Button(button_style='success', description='Нормализировать', style=ButtonStyle())

<IPython.core.display.Javascript object>

In [31]:
%blockrun button_normalize

In [32]:
#нормализация
normalize()

In [33]:
def marksearch(sost):
    value_dict.update({f'{sost}': {}})
    coun = query(inst,f'CALC1:PAR:COUN?')
    for trace in range(1,int(coun)+1):
        
        write(inst,f'CALC:PAR{trace}:SEL')
        write(inst, f'CALC1:TRAC{trace}:MARK1:STAT ON')
        write(inst, f'CALC1:TRAC{trace}:MARK2:STAT ON')
        write(inst, f'CALC:MARK1:FUNC:TYPE MAX')
        write(inst, f'CALC:MARK2:FUNC:TYPE MIN')
        
        write(inst, f'CALC:MARK1:FUNC:EXEC')
        write(inst, f'CALC:MARK2:FUNC:EXEC')
        
        max_value = abs(query_ascii_values(inst, f'CALC:MARK1:Y?')[0])
        min_value = abs(query_ascii_values(inst, f'CALC:MARK2:Y?')[0])
        if max_value > min_value:
            value_dict[f'{sost}'].update({f'{trace}': max_value})
            if(sost == 'before'):
                value_dict[f'table'].update({f'{trace}': max_value})
        else:
            value_dict[f'{sost}'].update({f'{trace}': min_value})
            if(sost == 'before'):
                value_dict[f'table'].update({f'{trace}': min_value})

In [34]:
from IPython.display import HTML, display

def tablehtml(): 
    #создаем строку для ввода в html страницу
    htmlstr = "<table border='1' ><tr><td>Порт</td><td>Отклонение</td><td>Максимум, дБ</td></tr><tr>"
    value_dict.update()
    MAX = -1
    for trace in value_dict["table"].keys():
        if value_dict["table"][f'{trace}'] > MAX:
            MAX = value_dict["table"][f'{trace}']
    for trace in range(1,len(value_dict["table"].keys())+1):
        trace_value = value_dict["table"][f'{trace}']
        htmlstr += f'<td>{trace}</td><td>{trace_value}</td>'
        if trace == 1:
            htmlstr += f'<td rowspan={len(value_dict["table"].keys())+1}>{MAX}</td>'
        htmlstr += "</tr><tr>"
    
    with open('html_table.html', 'w', encoding='utf-8') as fh1: 
         fh1.write(htmlstr)
    out.value = htmlstr



In [35]:
def on_button_clicked(b, rs_="some_default_string"):
    port_data = rs_.split(' ')
    value_dict['table'][f'{port_data[1]}'] = value_dict[f'{port_data[0]}'][f'{port_data[1]}']
    tablehtml()

In [36]:
marksearch("before")
#change
marksearch("after")

tablehtml()
#gridbutton(int(query(inst,f'CALC1:PAR:COUN?')))
b1 = widgets.Button(description="После 1", layout=widgets.Layout(display='flex',width='auto', height='auto'))
b1.on_click(functools.partial(on_button_clicked, rs_=f'after 1'))
b2 = widgets.Button(description="После 2", layout=widgets.Layout(display='flex',width='auto', height='auto'))
b2.on_click(functools.partial(on_button_clicked, rs_=f'after 2'))
b3 = widgets.Button(description="После 3", layout=widgets.Layout(display='flex',width='auto', height='auto'))
b3.on_click(functools.partial(on_button_clicked, rs_=f'after 3'))
b4 = widgets.Button(description="После 4", layout=widgets.Layout(display='flex',width='auto', height='auto'))
b4.on_click(functools.partial(on_button_clicked, rs_=f'after 4'))
display(b1)
display(b2)
display(b3)
display(b4)
display(out)

Button(description='После 1', layout=Layout(display='flex', height='auto', width='auto'), style=ButtonStyle())

Button(description='После 2', layout=Layout(display='flex', height='auto', width='auto'), style=ButtonStyle())

Button(description='После 3', layout=Layout(display='flex', height='auto', width='auto'), style=ButtonStyle())

Button(description='После 4', layout=Layout(display='flex', height='auto', width='auto'), style=ButtonStyle())

HTML(value="<table border='1' ><tr><td>Порт</td><td>Отклонение</td><td>Максимум, дБ</td></tr><tr><td>1</td><td…